In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pydantic.dataclasses import dataclass, Field
import dotenv
import interlab
import langchain
from interlab import actor, context
from interlab.context import Context, with_context
from interlab.lang_models.web_console import WebConsoleModel
from interlab.ext.pyplot import capture_figure
from interlab.environment.commgame import CommunicateAndPlayGame
from matplotlib import pyplot as plt
dotenv.load_dotenv()

True

In [2]:
import langchain
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

In [3]:
console_actor = actor.WebConsoleActor("MyConsoleActor")
console_actor

<WebConsoleActor MyConsoleActor>

In [20]:
PROMPT = """You are {name}. We are playing a repeated game where each player may choose one of the following actions: "cooperate" or "defect".
Before the each we can simulatenously send a message to each other. After that we both secretely choose one of action.
If we both play "cooperate" then we get $3 each. If I play "defect" and the opponent plays "cooperate" then I get $5 and they get nothing. If they play "defect" when I play "cooperate" then they get $5 and I get nothing.
If we both play "defect" then we get $1 each.
"""

e35 = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')

pa = actor.OneShotLLMActor("Alice", e35, PROMPT.format(name="Alice"))
pb = actor.OneShotLLMActor("Bob", e35, PROMPT.format(name="Bob"))
#pb = console_actor

actions = ["cooperate", "defect"]

payoffs = np.array(
    [
        [[3, 3], [0, 5]],
        [[5, 0], [1, 1]],
    ]
)

game = CommunicateAndPlayGame(actors=[pa, pb], n_rounds=3, action_names=actions, payoff_matrix=payoffs)
plt.ioff()
with Context("game") as ctx:
    game.run_until_end(verbose=True)
    ctx.set_result({
        "history": game.history,
        "payoffs": capture_figure(game.payoff_chart())
    })
ctx.display()

Step 0
Step 1
Step 2


In [35]:
game.history

[[<ActionEnum.cooperate: 'cooperate'>, <ActionEnum.cooperate: 'cooperate'>],
 [<ActionEnum.cooperate: 'cooperate'>, <ActionEnum.cooperate: 'cooperate'>],
 [<ActionEnum.cooperate: 'cooperate'>, <ActionEnum.cooperate: 'cooperate'>]]